# Symmetries in Bell nonlocality, part 1

For a compact introduction to representation theory, see the [book of Jean-Pierre Serre](https://link.springer.com/book/10.1007/978-1-4684-9458-7).

Follow the [installation instructions](https://replab.github.io/replab/docs/installation.html).

Then add the RepLAB path. The path below is the one for the machine that produced this notebook! Replace by your own. Or, better, move to the root RepLAB folder and just run the `replab_init` script.

In [1]:
addpath([pwd, '/../../..']);
replab_init('verbose', 0);

## Symmetries of the outcomes of a measurement

We start by studying the symmetries of a single measurement on a quantum system. Let us say that the measurement has outcomes $a = 1, 2, \ldots, k$.

### Symmetry group

We are interested in relabelings of these outcomes; those relabelings are represented by permutations. The group of all permutations of $k$ elements is the symmetric group of degree $k$, written $S_k$.

For the motivation, see our [paper on the symmetries of nonlocality in the context of device-independence](https://arxiv.org/abs/1610.01833). See also the [description of the symmetric group on Wikipedia](https://en.wikipedia.org/wiki/Symmetric_group).

In particular, there are different ways of writing elements of a symmetric group. RepLAB chooses to write down the permutation as the image row vector. For $\pi \in S_5$, we write $\pi = [\pi(1), \pi(2), \pi(3), \pi(4), \pi(5)]$ (using square brackets to distinguish this notation from the cycle notation), which corresponds to the *one-line notation* of https://en.wikipedia.org/wiki/Permutation#Notations .

For the group axioms, Wikipedia has a [compact discussion](https://en.wikipedia.org/wiki/Group_(mathematics)) . When composing permutations, one needs to decide, for $\pi, \sigma \in S_k$, how to compute the product $\pi \cdot \sigma$. We make the choice of having $(\pi \cdot \sigma)(a) = \pi ( \sigma (a))$, which corresponds to a left action, i.e. $\sigma$ is applied first. See https://en.wikipedia.org/wiki/Permutation#Composition_of_permutations for details.

This choice of composition corresponds to standard notation used in physics; however computational group theory uses a right action convention, most of the time using the *exponent notation*. Those notation choices should not concern the RepLAB user, as we made sure that the software uses consistent notation.

Below, we construct the symmetric group $S_5$. The object `S5` we construct knows how to compose permutations, when they are written using row vectors of integers in Matlab/Octave. The composition example is the one from [Wikipedia](https://en.wikipedia.org/wiki/Symmetric_group#Multiplication), and we check that we recover the same results.

In [2]:
S5 = replab.SymmetricGroup(5); % or shorter, S5 = replab.S(5)
f = [3 2 1 5 4]
g = [2 5 4 3 1]
fg = S5.compose(f, g)

    SymmetricGroup at line 46 column 17



f =

   3   2   1   5   4



g =

   2   5   4   3   1



fg =

   2   4   5   1   3



Note that permutations carry the size of their domain with them: the permutation $[2, 3, 1, 4]$ is not the same as the permutation $[2, 3, 1]$ even if both have the same action on $1,2,3$ and the first one leaves $4$ invariant.

**Exercice**: for fun, try to run `S5.order`, `S5.inverse(f)`, `S5.elements`, `S5.sample`. Construct `S50 = replab.SymmetricGroup(50)`. Compute `S50.order`, and try `S50.elements`.

### Representation

The distribution of outcomes of the measurement above can be represented by the distribution $P_\text{A}(a)$. We can also represent the distribution as a vector $\vec{P}_\text{A} \in V = \mathbb{R}^5$, with the following enumeration of coefficients: $\vec{P}_\text{A}^\top = \left ( P_\text{A}(1), P_\text{A}(2),  P_\text{A}(3),  P_\text{A}(4) ,  P_\text{A}(5) \right )$ . This action of elements of $S_5$ on $V = \mathbb{R}^5$ is encoded as a [group representation](https://en.wikipedia.org/wiki/Representation_theory#Definition); in RepLAB we call it the *defining representation* of $S_5$.

In [3]:
rep5 = S5.naturalRep
rep5.image(g)
rep5.image(f)*rep5.image(g) - rep5.image(S5.compose(f, g)) % check the representation composition axiom

rep5 =


Orthogonal reducible representation
   dimension: 5
       field: 'R'
       group: Symmetric group acting on 5 elements
   isUnitary: true
preimages{1}: [2, 3, 4, 5, 1]
   images{1}: 5 x 5 double
preimages{2}: [2, 1, 3, 4, 5]
   images{2}: 5 x 5 double



ans =

   0   0   0   0   1
   1   0   0   0   0
   0   0   0   1   0
   0   0   1   0   0
   0   1   0   0   0



ans =

   0   0   0   0   0
   0   0   0   0   0
   0   0   0   0   0
   0   0   0   0   0
   0   0   0   0   0



### Invariant subspaces and subrepresentations

Let $G$ be a group, and $\rho: G \to GL(V)$ a representation of $G$ on a vector space $V$. The space $GL(V)$ designates invertible linear maps from $V$ to $V$. In RepLAB, the vector space $V$ is defined either over the real or complex numbers.

An invariant subspace of $V$ is a subspace $W \subseteq V$ that stays invariant under the action of $\rho$. Formally, we want for all $\vec{w} \in W$ and $g \in G$ that $\rho_g ~ \vec{w} \in W$.

Any representation $V$ has two subrepresentations: the empty subspace $0$ and $V$ itself.

For any permutation representation like `rep5` above, the subspace spanned by the vector of all ones $(1,1,1,1,1)^\top$ is a subrepresentation: any vector of the form $(x,x,x,x,x)^\top\in\mathbb{R}^5$ stays invariant under permutation of coefficients.

The representations we construct are unitary, that is $\rho_g^+ = \rho_g^{-1} = \rho_{g^{-1}}$. Most of the literature assumes unitarity "without loss of generality", as a finite dimensional nonunitary representation for a compact group can always be made unitary after a change of basis.

With the other assumptions we make, this means that the orthogonal complement to $(1,1,1,1,1)^\top$ is also an invariant subspace.

By restricting a representation $\rho$ to one of its invariant subspaces, one then defines a subrepresentation.

A representation that has no invariant subspaces except $0$ and the full space $V$ is *irreducible*. The decomposition of a representation into irreducible representations can be obtained by recursively decomposing subrepresentations until the components are irreducible; obtaining that decomposition is pretty tricky, but luckily RepLAB automates that.

We obtain this decomposition and consider the first irreducible subrepresentation: the vector of all ones. Note that RepLAB recovers a "nice" algebraic expression for the basis of that subspace if we apply the `nice` method on the decomposition; this works in a few cases, the rest of the time that basis will be numerical/approximate to floating-point precision. We also check how the representation looks in that invariant subspace.

In [4]:
% The floating point approximate decomposition
D5 = rep5.decomposition

D5 =

Orthogonal reducible representation
        dimension: 5
            field: 'R'
            group: Symmetric group acting on 5 elements
        isUnitary: true
           parent: Orthogonal reducible representation
basis(1,'double'): [0.44721; 0.44721; 0.44721; 0.44721; 0.44721]
basis(2,'double'): [-0.028769; 0.87242; -0.30454; -0.26956; -0.26956]
basis(3,'double'): [-8.9605e-18; 1.0576e-18; 2.0581e-17; 0.70711; -0.70711]
basis(4,'double'): [0.88557; -0.19719; -0.34178; -0.1733; -0.1733]
basis(5,'double'): [-0.12221; 0.00223; -0.7684; 0.44419; 0.44419]
     component(1): Isotypic component R(1) (trivial)
     component(2): Isotypic component R(4) (nontrivial)



In [5]:
% Recovering nice coefficients for the basis
N5 = D5.nice

N5 =


Real reducible representation
       dimension: 5
           field: 'R'
           group: Symmetric group acting on 5 elements
       isUnitary: false
          parent: Orthogonal reducible representation
basis(1,'exact'): [1/5; 1/5; 1/5; 1/5; 1/5]
basis(2,'exact'): [4/5; -1/5; -1/5; -1/5; -1/5]
basis(3,'exact'): [-1/5; 4/5; -1/5; -1/5; -1/5]
basis(4,'exact'): [-1/5; -1/5; 4/5; -1/5; -1/5]
basis(5,'exact'): [-1/5; -1/5; -1/5; 4/5; -1/5]
    component(1): Isotypic component R(1) (trivial)
    component(2): Isotypic component R(4) (nontrivial)



In [6]:
% Images in the subrepresentations
subrep5a = D5.irrep(1) % this is the first irreducible component
subrep5a.image(S5.generator(1))
subrep5a.image(S5.generator(2))

subrep5a =

Orthogonal trivial irreducible real-type subrepresentation
        dimension: 1
            field: 'R'
            group: Symmetric group acting on 5 elements
        isUnitary: true
           parent: Orthogonal reducible representation
basis(1,'double'): [0.44721; 0.44721; 0.44721; 0.44721; 0.44721]



ans = 1


ans = 1


We note that both generators of $S_5$ look like the identity matrix of dimension $1$ in that subrepresentation. Thus, any element of $S_5$ corresponds to the identity matrix for that subrepresentation. Such a representation with an identity image is the *trivial representation*.

We move now to the other subrepresentation.

In [7]:
subrep5b = D5.irrep(2)
subrep5b.image(S5.generator(1))
subrep5b.image(S5.generator(2))

subrep5b =

Orthogonal fully nontrivial irreducible real-type subrepresentation
        dimension: 4
            field: 'R'
            group: Symmetric group acting on 5 elements
        isUnitary: true
           parent: Orthogonal reducible representation
basis(1,'double'): [-0.028769; 0.87242; -0.30454; -0.26956; -0.26956]
basis(2,'double'): [-8.9605e-18; 1.0576e-18; 2.0581e-17; 0.70711; -0.70711]
basis(3,'double'): [0.88557; -0.19719; -0.34178; -0.1733; -0.1733]
basis(4,'double'): [-0.12221; 0.00223; -0.7684; 0.44419; 0.44419]



ans =

  -0.128277  -0.170261   0.976467  -0.032685
  -0.024736  -0.500000  -0.119132  -0.857436
  -0.431720  -0.748735  -0.171439   0.472887
  -0.892497   0.400508  -0.054116  -0.200284



ans =

   1.8787e-01  -2.6591e-17   9.7577e-01  -1.1214e-01
  -2.6591e-17   1.0000e+00   2.0591e-18   6.1048e-19
   9.7577e-01   2.0591e-18  -1.7238e-01   1.3474e-01
  -1.1214e-01   6.1048e-19   1.3474e-01   9.8451e-01



Now the generators of $S_5$ look like they are doing something in that subrepresentation! That representation is actually the [standard representation](https://groupprops.subwiki.org/wiki/Standard_representation) of $S_5$.

### Physical interpretation

The physical interpretation of invariant subspaces is that they correspond to properties that are preserved under symmetry.

In the present case, the split into the two subrepresentations highlight a property of probability distributions invariant under relabeling of outcomes, the *normalization* of the distribution.

### The original representation in the basis that exhibits the decomposition

The matrix $B$ is the change of basis that block diagonalizes the representation. Here, we can show that `rep5` has a 1x1 and 4x4 block structure in the relevant basis.

In [8]:
B = D5.basis
inv(B) * rep5.image(S5.generator(1)) * B
inv(B) * rep5.image(S5.generator(2)) * B

B =

   4.4721e-01  -2.8769e-02  -8.9605e-18   8.8557e-01  -1.2221e-01
   4.4721e-01   8.7242e-01   1.0576e-18  -1.9719e-01   2.2300e-03
   4.4721e-01  -3.0454e-01   2.0581e-17  -3.4178e-01  -7.6840e-01
   4.4721e-01  -2.6956e-01   7.0711e-01  -1.7330e-01   4.4419e-01
   4.4721e-01  -2.6956e-01  -7.0711e-01  -1.7330e-01   4.4419e-01



ans =

   1.0000e+00   2.5922e-17  -1.5851e-17   6.2440e-17   5.6745e-17
   1.2798e-16  -1.2828e-01  -1.7026e-01   9.7647e-01  -3.2685e-02
  -1.8731e-17  -2.4736e-02  -5.0000e-01  -1.1913e-01  -8.5744e-01
  -1.7479e-16  -4.3172e-01  -7.4874e-01  -1.7144e-01   4.7289e-01
   2.1583e-17  -8.9250e-01   4.0051e-01  -5.4116e-02  -2.0028e-01



ans =

   1.0000e+00  -1.1286e-16  -1.5851e-17   6.2440e-17   1.2336e-18
   1.0517e-16   1.8787e-01  -1.0332e-17   9.7577e-01  -1.1214e-01
  -1.7174e-17  -5.3484e-18   1.0000e+00  -1.5019e-19  -1.7977e-17
  -1.8164e-16   9.7577e-01   2.1685e-17  -1.7238e-01   1.3474e-01
   2.0031e-17  -1.1214e-01   3.3128e-17   1.3474e-01   9.8451e-01



## Another example

To continue this discussion, we consider a measurement with two outcomes $a=1,2$. The symmetry group is now $S_2$, and has two elements: identity $[1,2]$ and flip $[2,1]$.

In [9]:
S2 = replab.SymmetricGroup(2)
S2.elements


    SymmetricGroup at line 46 column 17


S2 =


Symmetric group acting on 2 elements
            identity: [1, 2]
generator(1 or 'x1'): [2, 1]
    recognize.source: Cyclic group C(2) of order 2 < x | x^2 = 1 >



ans =
{
  [1,1] =

     1   2

  [1,2] =

     2   1

}



The defining representation has two images: the identity $\begin{pmatrix}1 & 0\\0 & 1\end{pmatrix}$ and the flip $\begin{pmatrix}0 & 1\\1 & 0 \end{pmatrix}$.

In [10]:
rep2 = S2.naturalRep
rep2.image([1,2])
rep2.image([2,1])

rep2 =

Orthogonal reducible representation
   dimension: 2
       field: 'R'
       group: Symmetric group acting on 2 elements
   isUnitary: true
preimages{1}: [2, 1]
   images{1}: [0, 1; 1, 0]



ans =

   1   0
   0   1



ans =

   0   1
   1   0



Now, one can check that the two subrepresentations correspond to vector spaces spanned by $(1,1)^\top$ and $(1,-1)^\top$ respectively.

In [11]:
D2 = rep2.decomposition.nice
D2.irrep(1).basis
D2.irrep(2).basis

D2 =

Real reducible representation
       dimension: 2
           field: 'R'
           group: Symmetric group acting on 2 elements
       isUnitary: false
          parent: Orthogonal reducible representation
basis(1,'exact'): [1/2; 1/2]
basis(2,'exact'): [1/2; -1/2]
    component(1): Isotypic component R(1) (trivial)
    component(2): Isotypic component R(1) (nontrivial)



ans =

   0.5000
   0.5000



ans =

   0.5000
  -0.5000



Up to a multiplicative factor, these two vectors extract two quantities: $(1,1) \cdot \vec{P}$ computes the normalization, while $(1,-1) \cdot \vec{P}$ computes the correlator $P(1) - P(2)$ often used in quantum information for systems with binary outcomes.

Note that the flip operation leaves $(1,1)$ invariant, while it changes the sign of $(1,-1)$. Accordingly, the subrepresentation in the subspace spanned by $(1,-1)$ is called the *sign representation*.

## Conditional probability distributions

We now move to the study of conditional probability distributions $P_{A|X}(a|x)$. For simplicity, we consider binary outputs $a=1,2$ and binary inputs $x=1,2$. We enumerate the coefficients in a vector $\vec{P} = (P(1|1), P(2|1), P(1|2), P(2|2))^\top$.

Different operations are available:

- Permutation of inputs, which will permute coefficients of $\vec{P}$ according to $g_1 = [3,4,1,2]$.

- Permutation of outputs of the first measurement, which will permute $\vec{P}$ according to $g_2 = [2,1,3,4]$.

- The permutation of outputs of the second measurement is achieved by conjugating $g_2$ by $g_1$, i.e. by $g_1 ~ g_2 ~ g_1$, so we do not need to provide that group generator explicitly.

We then construct the group $G=\left <g_1, g_2 \right>$ and its defining representation, which we decompose into irreducible subrepresentations.

In [12]:
g1 = [3,4,1,2];
g2 = [2,1,3,4];
generators = {g1 g2}; % arrays of things in Matlab are cell arrays
S4 = replab.SymmetricGroup(4);
G = S4.subgroup(generators) % construction as a subgroup of a generic parent group
G.elements
Grep = G.naturalRep

    SymmetricGroup at line 46 column 17



G =


Permutation group acting on 4 elements of order 8
            identity: [1, 2, 3, 4]
generator(1 or 'x1'): [3, 4, 1, 2]
generator(2 or 'x2'): [2, 1, 3, 4]
    recognize.source: Dihedral group of order 8



ans =
{
  [1,1] =

     1   2   3   4

  [1,2] =

     1   2   4   3

  [1,3] =

     2   1   3   4

  [1,4] =

     2   1   4   3

  [1,5] =

     3   4   1   2

  [1,6] =

     3   4   2   1

  [1,7] =

     4   3   1   2

  [1,8] =

     4   3   2   1

}



Grep =


Orthogonal reducible representation
   dimension: 4
       field: 'R'
       group: Permutation group acting on 4 elements of order 8
   isUnitary: true
preimages{1}: [3, 4, 1, 2]
   images{1}: [0, 0, 1, 0; 0, 0, 0, 1; 1, 0, 0, 0; 0, 1, 0, 0]
preimages{2}: [2, 1, 3, 4]
   images{2}: [0, 1, 0, 0; 1, 0, 0, 0; 0, 0, 1, 0; 0, 0, 0, 1]



In [13]:
% Let's decompose this representation into irreducible subrepresentations
Gdec = Grep.decomposition.nice
I1 = Gdec.irrep(1)
I2 = Gdec.irrep(2)
I3 = Gdec.irrep(3)

Gdec =

Real reducible representation
       dimension: 4
           field: 'R'
           group: Permutation group acting on 4 elements of order 8
       isUnitary: false
          parent: Orthogonal reducible representation
basis(1,'exact'): [1/4; 1/4; 1/4; 1/4]
basis(2,'exact'): [1/4; 1/4; -1/4; -1/4]
basis(3,'exact'): [1/2; -1/2; 0; 0]
basis(4,'exact'): [0; 0; 1/2; -1/2]
    component(1): Isotypic component R(1) (trivial)
    component(2): Isotypic component R(1) (nontrivial)
    component(3): Isotypic component R(2) (nontrivial)



I1 =

Real trivial irreducible real-type subrepresentation
       dimension: 1
           field: 'R'
           group: Permutation group acting on 4 elements of order 8
       isUnitary: false
          parent: Orthogonal reducible representation
basis(1,'exact'): [1/4; 1/4; 1/4; 1/4]



I2 =

Real fully nontrivial irreducible real-type subrepresentation
       dimension: 1
           field: 'R'
           group: Permutation group acting on 4 elements of order 8
       isUnitary: false
          parent: Orthogonal reducible representation
basis(1,'exact'): [1/4; 1/4; -1/4; -1/4]



I3 =

Real fully nontrivial irreducible real-type subrepresentation
       dimension: 2
           field: 'R'
           group: Permutation group acting on 4 elements of order 8
       isUnitary: false
          parent: Orthogonal reducible representation
basis(1,'exact'): [1/2; -1/2; 0; 0]
basis(2,'exact'): [0; 0; 1/2; -1/2]



What are the irreps we obtain? We have:

- A first subspace encoding the overall normalization $(1,1,1,1) \cdot \vec{P} = 2$, by definition of $\sum_{a} P(a|x) = 1$.

- A second subspace encoding the difference in normalization between the condition $x=1$ and the condition $x=2$: $(1,1,-1,-1) \cdot \vec{P} = 0$.

- The third subspace, after a change of basis vectors, corresponds to the binary correlators $(1,-1,0,0) \cdot \vec{P} = P(1|1) - P(2|1)$ and $(0,0,1,-1) \cdot \vec{P} = P(1|2) - P(2|2)$.

We leave, as an exercice, the construction of the symmetry group of the CHSH scenario using its action on the joint conditional distribution $P_{AB|XY}(ab|xy)$ -- one should recover the results of [our paper](https://arxiv.org/abs/1610.01833).

## A final word of warning

The presentation above glossed over an important concept: the decomposition of a representation into irreducible subrepresentations is essentially unique, up to the degeneracy afforded by multiplicities. Indeed, when several copies of the same representation appear, the decomposition into irreducible subrepresentation is not exactly unique; refer to the end of Chapter 2 of the book by Jean-Pierre Serre cited in the preface.

In particular, the method `dec.irrep(i)` needs to be replaced by `dec.irrep(i,j)` where $i$ is the index of the irreducible representation type, and $j$ is the copy index.

Also, if one works using real representations, there could be a difference in the irreducible decomposition when the representation is treated as a complex one. This is because the real field is not algebraically closed. Over the reals, there are three kinds of irreducible representations: RepLAB identifies and puts them in a canonical representation. Not to worry: representation of symmetric groups and derivatives (direct products and the like) are decomposed the same way over the real and complex numbers.